In [ ]:
# Keras 2.3.1, tensorflow 2.0 환경에서
# AttributeError: '_thread._local' object has no attribute 'value' 발생하면
# Keras 2.3.0 이후 버전은 2.2.5 버전으로 재설치 or app.run(threaded=False)으로 해결

# Keras 2.2.5를 사용하려면 tensorflow 1.14 버전으로 재설치 해야되고
# 아래와 같은 이슈가 발생하며
# AttributeError: '_thread._local' object has no attribute 'value'
# model을 로드하고 model._make_predict_function() 추가하여 해결

In [1]:
from flask import Flask, jsonify, request
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import h5py
from PIL import Image
import numpy as np
import collections 
import io
import os
import glob
import json

# 카테고리 지정하기
categories = [x.split('\\')[1] for x in glob.glob("./image/new/*")]
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 64 
image_h = 64
#np.load에 allow_pikle 추가
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# 데이터 불러오기 --- (※1)
X_train, X_test, y_train, y_test = np.load("./image/fish.npy")
# 데이터 정규화하기
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)
# 모델 구축하기 --- (※2)
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # --- (※3) 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
#model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

# 모델 훈련하기 --- (※4)
hdf5_file = "./image/fish-model.hdf5"
if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
    print("hdf5_file loaded")
else:
    model.fit(X_train, y_train, batch_size=32, epochs=50)
    model.save_weights(hdf5_file)
    
# AttributeError: '_thread._local' object has no attribute 'value' 해결용
# model._make_predict_function()

Using TensorFlow backend.


X_train shape: (252, 64, 64, 3)


C:\Users\SkeldTcan\anaconda3\envs\hong\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`


hdf5_file loaded


C:\Users\SkeldTcan\anaconda3\envs\hong\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
C:\Users\SkeldTcan\anaconda3\envs\hong\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [2]:
# 이미지를 읽어 결과를 반환하는 함수
def get_prediction(image_bytes):
    # 이미지 저장
    image = Image.open(io.BytesIO(image_bytes))

    X = []    
    img = image.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    X.append(data)
    X = np.array(X)
    y = model.predict(X)
    
    # 물고기별 확률 계산
    total = 0
    for i in y[0]:
        if i>0:
            total += i

    # 딕셔너리로 저장
    fish = collections.OrderedDict()
    for k, v in zip(categories, y[0]):
        if v>0:
            fish[k] = v/total
        else:
            fish[k] = 0
    fish = collections.OrderedDict(sorted(fish.items(), key=lambda x: -x[1]))
    return fish

app = Flask(__name__)

@app.route('/', methods=['POST'])
def predict():
    if request.method == 'POST':
        # 이미지 바이트 데이터 받아오기
        file = request.files['file']
        image_bytes = file.read()

        # 분류 결과 확인 및 클라이언트에게 결과 반환
        class_name = get_prediction(image_bytes=image_bytes)
        
        result = "{"
        for k, v in class_name.items():
            if v == 0:
                break
            result += f"'{k}':{v*100:.3f},"
        result = result[:-1] + "}"
        
        print(class_name)
        print(result)
        return json.dumps(result)

In [ ]:
# 모든 호스트에 대해 접속 허용
app.run(threaded=False, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.0.1 - - [25/Mar/2021 13:47:36] "POST / HTTP/1.1" 200 -


OrderedDict([('golden mandarin fish', 0.622569933420389), ('catfish', 0.1854796785731574), ('red sanpper', 0.09607049686347378), ('carpfish', 0.09587989114297979), ('flatfish', 0)])
{'golden mandarin fish':62.257,'catfish':18.548,'red sanpper':9.607,'carpfish':9.588}
